In [2]:
import gym
from gym import error
from gym.utils import closer
from PIL import Image
import torch
from pypylon import pylon
import numpy as np
import cv2
from ctypes import *
import glob
import matplotlib.pylab as pl
import matplotlib.pyplot as plt
import gym
from gym import error
from gym.utils import closer
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common import make_vec_env
from stable_baselines import A2C, PPO2
from gym import spaces
from pypylon import pylon
import stable_baselines
import time
from matplotlib import cm
from stable_baselines.common.schedules import LinearSchedule
%matplotlib inline

env_closer = closer.Closer()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
class CorningEnv(gym.Env):
    """The main OpenAI Gym class. It encapsulates an environment with
    arbitrary behind-the-scenes dynamics. An environment can be
    partially or fully observed.
    The main API methods that users of this class need to know are:
        step
        reset
        render
        close
        seed
    And set the following attributes:
        action_space: The Space object corresponding to valid actions
        observation_space: The Space object corresponding to valid observations
        reward_range: A tuple corresponding to the min and max possible rewards
    Note: a default reward range set to [-inf,+inf] already exists. Set it if you want a narrower range.
    The methods are accessed publicly as "step", "reset", etc...
    
    """
    metadata = {'render.modes': ['rgb_array']}
    

    
    def __init__(self, camera, 
                     GainRaw = 36, 
                     ExposureTimeRaw = 20000, 
                     AcquisitionFrameRateAbs = 10,
                     Width = 417,
                     Height = 404,
                     threshold = 85):
        
        ##############camera##############
        
        # Create an instant camera object with the camera device found first.
        #self.camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
        #self.camera.Open()
        
        self.camera = camera
        
        # https://docs.baslerweb.com/features
        self.camera.GainRaw.Value = GainRaw
        self.camera.ExposureTimeRaw.Value = ExposureTimeRaw
        self.camera.AcquisitionFrameRateAbs.Value = AcquisitionFrameRateAbs
        
        # get width and height from camera for making obs space
        self.camera.Width.Value = Width
        self.camera.Height.Value = Height
        print(self.camera.Width.GetValue(), self.camera.Height.GetValue())
        
        
        ##############lens###############
        # corning lib
        self.lib = cdll.LoadLibrary(r"C:\Users\CIG\Documents\MATLAB\ComCasp64.dll")
        #Check if Maxim driver dll is loaded
        eCOMCaspErr = getattr(self.lib,'Casp_OpenCOM')
        print('eCOMCaspErr:', eCOMCaspErr(), self.lib.Casp_OpenCOM())
        
        
        ##############env action and observation##############
        # Focus Voltage
#         self.action_space = spaces.Box(low=np.array([24.0]), 
#                                        high=np.array([69.0]), 
#                                        shape=(1,), 
#                                        dtype = np.float32)
        self.action_space = spaces.MultiDiscrete([69, 99])
        # image
        self.observation_space = spaces.Box(low=0, 
                                            high=255, 
                                            shape=(Height, Width, 3),
                                            dtype = np.uint8)
        
        
        ##############for grabbing images from camera##############
        # code from https://github.com/basler/pypylon/blob/master/samples/opencv.py
        img = pylon.PylonImage()
        self.converter = pylon.ImageFormatConverter()
        
        # converting to opencv bgr format
        self.converter.OutputPixelFormat = pylon.PixelType_BGR8packed
        self.converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned
        
        ##############YOLO##############
        self.model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', 
                                         force_reload=True, 
                                         pretrained=True)
        
        ########################################################
        self.threshold = threshold
        self.file_path = r'C:\Users\CIG\Documents\imgs_RL_floatV'
        self.i = 0
        time.sleep(1)
        
#     def compute_reward(self, achieved_goal, desired_goal, info):
#         """Compute the step reward. This externalizes the reward function and makes
#         it dependent on a desired goal and the one that was achieved. If you wish to include
#         additional rewards that are independent of the goal, you can include the necessary values
#         to derive it in 'info' and compute it accordingly.
#         Args:
#             achieved_goal (object): the goal that was achieved during execution
#             desired_goal (object): the desired goal that we asked the agent to attempt to achieve
#             info (dict): an info dictionary with additional information
#         Returns:
#             float: The reward that corresponds to the provided achieved goal w.r.t. to the desired
#             goal. Note that the following should always hold true:
#                 ob, reward, done, info = env.step()
#                 assert reward == env.compute_reward(ob['achieved_goal'], ob['goal'], info)
#         """
#         raise NotImplementedError
        
        
    def find_class(self, results, obj = 'car'):
        len_of_class = len(results.pred)
        pred_class = [int(results.pred[0][i].numpy()[5]) for i in range(len(results.pred[0]))]
        #list_of_yolo_classes = list(zip(results.names, range(0, len(results.names))))
        pred_names = [results.names[i] for i in pred_class]
        if obj in pred_names:
            index = pred_names.index(obj)
            return results.pred[0][index].numpy()
        else:
            return np.array([])
        
    def focus_value(self, im):
        # Calculate the gradient
        sobelx = cv2.Sobel(np.float32(im), cv2.CV_64F ,1 , 0, ksize=5)
        sobely = cv2.Sobel(np.float32(im),cv2.CV_64F,0,1,ksize=5)

        abs_sobel_x = cv2.convertScaleAbs(sobelx) # converting back to uint8
        abs_sobel_y = cv2.convertScaleAbs(sobely)
        #print(abs_sobel_x )

        # Combine the two gradients with equal weight
        dst = cv2.addWeighted(abs_sobel_x,0.5,abs_sobel_y,0.5,0)
        #print(dst)

        # Calculate the average gradient for the image
        # I convert it to a numpy array for ease of calculation
        return pl.asarray(dst).mean()
    
    def lens_movement(self, action):
        x = c_double(action)
        self.lib.Casp_SetFocusVoltage(x)
        time.sleep(2)
        
    def grab_image(self):
        #https://github.com/basler/pypylon/blob/master/samples/opencv.py
        self.camera.StartGrabbing()
        while 1:
            grabResult = self.camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)
            if grabResult.GrabSucceeded():
                # Access the image data
                image = self.converter.Convert(grabResult)
                image = image.GetArray()
#                 print(img[0])
#                 print('shape:', img.shape)
#                 plt.imshow(img)
                break
        self.camera.StopGrabbing()
        return image
    
    def step(self, action):
        
        action1 = 0
        action2 = action[1]
        
        if action[0] < 24:
            action1 = 24
        else:
            action1 = action[0]
            
        action = float("%s.%s"% (str(action1), str(action2)))

        self.lens_movement(action)
 
        camera_observation = self.grab_image()
        prediction_yolo = self.model_yolo(camera_observation)
        final_result_yolo = self.find_class(prediction_yolo)
        
        if final_result_yolo.size == 0:
            print('not detected')
            foc_value = -1
            betta = 0.5
            reward = -100-betta*self.i
            #reward = -100
            
        else:
            image = Image.fromarray(camera_observation)
            image = image.crop((final_result_yolo[0], 
                             final_result_yolo[1], 
                             final_result_yolo[2], 
                             final_result_yolo[3]))
            foc_value = self.focus_value(image)
            
        if foc_value > self.threshold: #or self.i == 20:
            done = True
            reward = 1
            print('------',True, action, self.i, '------')
        else:
            done = False
            reward = 1 - 0.001*(foc_value - self.threshold)**2
            #reward = -1
            print(False, action)
            
        im = Image.fromarray(camera_observation)
        filename = self.file_path + "\saved_pypylon_img_%s.png" % (str(action))
        im.save(filename)
        
        self.i+=1
        #print(camera_observation)
        camera_observation = camera_observation.astype(np.uint8)
            
        return camera_observation, reward, done, {}
  

    def reset(self):
        """Resets the environment to an initial state and returns an initial
        observation.
        Note that this function should not reset the environment's random
        number generator(s); random variables in the environment's state should
        be sampled independently between multiple calls to `reset()`. In other
        words, each call of `reset()` should yield an environment suitable for
        a new episode, independent of previous episodes.
        Returns:
            observation (object): the initial observation.
        """
        self.lens_movement(24.0)
        self.i = 0
        time.sleep(1)
        return self.grab_image()


    def close(self):
        """Override close in your subclass to perform any necessary cleanup.
        Environments will automatically close() themselves when
        garbage collected or when the program exits.
        """
        pass



In [4]:
# Create an instant camera object with the camera device found first.
camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
camera.Open()

In [5]:
env = CorningEnv(camera)

417 404
eCOMCaspErr: 0 0


Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\CIG/.cache\torch\hub\master.zip

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    156928  models.common.C3                        [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   

Adding autoShape... 


In [6]:
#from stable_baselines.common.env_checker import check_env
# It will check your custom environment and output additional warnings if needed
#check_env(env)
#env.action_space.low

not detected
False 24.8
not detected
False 35.33
not detected
False 24.94
not detected
False 37.2
not detected
False 61.88
not detected
False 26.56
not detected
False 53.91
not detected
False 24.83
not detected
False 38.4
not detected
False 24.79
not detected
False 24.96


In [7]:
# TIMESTEPS = 1000
# sched_LR = LinearSchedule(TIMESTEPS, 0.0005, 0.000001)

model = PPO2(MlpPolicy, env, 
             #learning_rate = sched_LR.value,
             verbose=1, 
             tensorboard_log = '')

Wrapping the env in a DummyVecEnv.



From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\common\tf_util.py:191: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\common\tf_util.py:200: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\common\policies.py:116: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.



From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\common\input.py:25: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



...

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


From C:\Users\CIG\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\ops\math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\ppo2\ppo2.py:206: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.



From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\ppo2\ppo2.py:240: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.



From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\ppo2\ppo2.py:242: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.



In [9]:
model.learn(total_timesteps=2000, tb_log_name="reward_2_floatV", reset_num_timesteps=False)

not detected
False 40.65
------ True 49.24 1 ------
not detected
False 44.57
------ True 49.31 1 ------
not detected
False 24.49
not detected
False 52.4
not detected
False 35.4
not detected
False 25.75
not detected
False 45.31
not detected
False 44.29
not detected
False 24.52
not detected
False 62.12
------ True 47.78 8 ------
not detected
False 24.35
not detected
False 24.83
------ True 48.97 2 ------
not detected
False 24.17
not detected
False 24.42
not detected
False 59.37
not detected
False 24.65
not detected
False 52.77
not detected
False 29.52
not detected
False 26.15
not detected
False 24.5
not detected
False 24.53
not detected
False 24.79
------ True 48.75 10 ------
not detected
False 24.67
not detected
False 24.3
------ True 47.26 2 ------
not detected
False 42.37
not detected
False 35.65
not detected
False 24.34
------ True 48.69 3 ------
------ True 47.5 0 ------
not detected
False 43.2
------ True 47.84 1 ------
not detected
False 52.35
not detected
False 24.54
not detected

not detected
False 44.71
not detected
False 60.23
------ True 49.4 3 ------
not detected
False 24.94
not detected
False 29.49
not detected
False 30.91
------ True 49.1 3 ------
------ True 49.23 0 ------
------ True 47.37 0 ------
not detected
False 44.38
------ True 48.57 1 ------
not detected
False 55.21
------ True 49.25 1 ------
not detected
False 67.44
not detected
False 38.61
------ True 47.78 2 ------
not detected
False 24.37
not detected
False 62.69
not detected
False 24.21
not detected
False 43.12
------ True 47.53 4 ------
not detected
False 24.19
not detected
False 61.85
not detected
False 24.65
not detected
False 43.49
not detected
False 27.37
not detected
False 42.31
not detected
False 24.57
not detected
False 46.84
not detected
False 33.3
------ True 48.66 9 ------
not detected
False 24.14
not detected
False 59.69
not detected
False 63.64
------ True 47.88 3 ------
not detected
False 67.16
------ True 49.52 1 ------
not detected
False 43.75
------ True 48.34 1 ------
not 

------ True 47.24 6 ------
not detected
False 53.62
------ True 49.53 1 ------
not detected
False 24.89
not detected
False 24.23
not detected
False 24.86
not detected
False 24.13
not detected
False 24.56
------ True 48.56 5 ------
------ True 47.25 0 ------
not detected
False 25.95
not detected
False 24.81
------ True 48.2 2 ------
------ True 48.29 0 ------
not detected
False 35.76
not detected
False 64.3
------ True 48.17 2 ------
not detected
False 30.63
not detected
False 59.19
not detected
False 24.15
not detected
False 24.39
not detected
False 37.6
not detected
False 64.35
------ True 49.68 6 ------
not detected
False 32.73
not detected
False 41.64
not detected
False 24.98
not detected
False 25.16
not detected
False 53.52
not detected
False 55.4
not detected
False 36.86
------ True 49.64 7 ------
not detected
False 24.8
------ True 48.97 1 ------
not detected
False 33.3
not detected
False 54.38
not detected
False 24.31
not detected
False 67.65
not detected
False 40.2
not detected

------ True 48.73 3 ------
------ True 48.26 0 ------
------ True 47.3 0 ------
------ True 47.79 0 ------
not detected
False 47.0
------ True 48.47 1 ------
not detected
False 34.86
not detected
False 68.97
not detected
False 24.15
not detected
False 66.68
not detected
False 24.11
------ True 48.87 5 ------
not detected
False 24.57
not detected
False 54.29
------ True 48.64 2 ------
not detected
False 24.56
not detected
False 41.9
not detected
False 41.75
not detected
False 24.93
not detected
False 24.34
------ True 47.86 5 ------
not detected
False 37.29
not detected
False 24.48
not detected
False 38.15
------ True 48.51 3 ------
not detected
False 40.85
------ True 48.29 1 ------
------ True 47.77 0 ------
not detected
False 58.4
------ True 47.36 1 ------
------ True 47.77 0 ------
not detected
False 24.82
------ True 47.92 1 ------
not detected
False 45.56
not detected
False 37.39
not detected
False 56.16
------ True 47.6 3 ------
not detected
False 38.32
not detected
False 40.13


------ True 47.33 1 ------
not detected
False 29.55
------ True 48.3 1 ------
------ True 47.3 0 ------
------ True 49.13 0 ------
not detected
False 24.82
------ True 47.74 1 ------
not detected
False 35.37
not detected
False 55.72
not detected
False 24.2
------ True 48.8 3 ------
not detected
False 46.21
------ True 47.6 1 ------
------ True 48.36 0 ------
------ True 47.9 0 ------
------ True 48.5 0 ------
------ True 48.8 0 ------
------ True 48.98 0 ------
------ True 48.58 0 ------
------ True 48.27 0 ------
-------------------------------------
| approxkl           | 0.0037320019 |
| clipfrac           | 0.01171875   |
| explained_variance | 0            |
| fps                | 0            |
| n_updates          | 9            |
| policy_entropy     | 7.4675255    |
| policy_loss        | -0.058208786 |
| serial_timesteps   | 1152         |
| time_elapsed       | 3.29e+03     |
| total_timesteps    | 3072         |
| value_loss         | 25.071346    |
------------------------

not detected
False 41.1
------ True 48.23 1 ------
------ True 47.3 0 ------
not detected
False 24.73
------ True 48.56 1 ------
------ True 47.85 0 ------
------ True 48.58 0 ------
------ True 48.49 0 ------
------ True 47.59 0 ------
------ True 48.53 0 ------
------ True 48.15 0 ------
not detected
False 33.74
not detected
False 60.25
------ True 48.17 2 ------
------ True 47.74 0 ------
------ True 48.9 0 ------
------ True 47.76 0 ------
------ True 48.54 0 ------
------ True 47.35 0 ------
------ True 47.75 0 ------
------ True 47.37 0 ------
------ True 48.94 0 ------
------ True 48.53 0 ------
------ True 48.93 0 ------
not detected
False 46.1
------ True 49.48 1 ------
not detected
False 51.41
------ True 47.38 1 ------
------ True 47.33 0 ------
------ True 47.52 0 ------
not detected
False 40.74
not detected
False 49.96
not detected
False 55.34
not detected
False 24.34
------ True 48.44 4 ------
------ True 47.63 0 ------
not detected
False 65.16
not detected
False 34.34
--

------ True 48.36 0 ------
------ True 47.95 0 ------
------ True 47.8 0 ------
------ True 47.64 0 ------
------ True 47.11 0 ------
------ True 48.14 0 ------
------ True 47.15 0 ------
------ True 47.47 0 ------
------ True 47.75 0 ------
------ True 47.89 0 ------
not detected
False 24.4
not detected
False 24.14
------ True 47.95 2 ------
------ True 48.45 0 ------
------ True 47.23 0 ------
------ True 47.75 0 ------
not detected
False 24.2
------ True 48.76 1 ------
------ True 47.2 0 ------
------ True 47.4 0 ------
------ True 49.64 0 ------
------ True 48.93 0 ------
------ True 47.95 0 ------
------ True 48.93 0 ------
------ True 47.64 0 ------
------ True 48.26 0 ------
------ True 47.97 0 ------
------ True 48.76 0 ------
------ True 48.2 0 ------
------ True 47.58 0 ------
------ True 47.52 0 ------
------ True 48.71 0 ------
------ True 48.37 0 ------
------ True 47.24 0 ------
------ True 47.3 0 ------
------ True 47.34 0 ------
------ True 48.67 0 ------
------ True 48

In [11]:
#model.save("model_reward_type2_floatV")

In [12]:
# Enjoy trained agent
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)

------ True 47.4 0 ------
------ True 47.58 1 ------
------ True 47.38 2 ------
------ True 47.91 3 ------
not detected
False 66.73
not detected
False 47.51
------ True 48.7 6 ------
------ True 48.34 7 ------
not detected
False 47.31
not detected
False 47.38
not detected
False 61.94
------ True 48.8 11 ------
not detected
False 41.86
------ True 48.39 13 ------
not detected
False 47.18
not detected
False 47.13
------ True 48.0 16 ------
not detected
False 47.56
not detected
False 47.43
------ True 48.19 19 ------
not detected
False 47.1
------ True 47.46 21 ------
------ True 48.64 22 ------
------ True 47.34 23 ------
------ True 48.12 24 ------
------ True 48.64 25 ------
not detected
False 24.23
------ True 47.4 27 ------
------ True 47.61 28 ------
------ True 47.5 29 ------
------ True 48.23 30 ------
------ True 47.1 31 ------
------ True 47.6 32 ------
------ True 47.87 33 ------
------ True 47.47 34 ------
------ True 47.97 35 ------
------ True 47.11 36 ------
------ True 47.

KeyboardInterrupt: 

In [9]:
#model.learn(total_timesteps=1000, tb_log_name="p_")

In [ ]:
#tensorboard --logdir=C:\\Users\\CIG\\Documents\\MATLAB

In [13]:
camera.Close()